In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [2]:
train_df=pd.read_csv('../input/nlp-getting-started/train.csv')
test_df=pd.read_csv('../input/nlp-getting-started/test.csv')

In [3]:
print(train_df.shape)
train_df.head()

(7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
print(test_df.shape)
test_df.head()

(3263, 4)


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


### We can see clearly that Keyword and Location Features Have Nan values and they are no use to predict our model, so we will drop these from both train and test dataframe

In [5]:
train_df.drop(columns=['id',"keyword",'location'],inplace=True,axis=0)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
test_df.drop(columns=['id','keyword','location'],inplace=True)
test_df.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


### Creating training and testing data

In [7]:
from sklearn.model_selection import train_test_split
x=train_df['text']
y=train_df['target']
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.2)

### Feature extraction through Count Vectorization scikit learn library

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
x_vector=vectorizer.fit_transform(x_train)

## Now done with feature  extraction now train our models

### Linear regression Model

In [9]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=500)
lr.fit(x_vector,y_train)
pred_y=lr.predict(vectorizer.transform(x_test))

In [10]:
from sklearn.metrics import accuracy_score
score_linear=accuracy_score(y_test,pred_y)
score_linear

0.8135259356533159

### Naive Bayes Model

In [11]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB(alpha=1)
nb.fit(x_vector,y_train)
y_pred=nb.predict(vectorizer.transform(x_test))
score_nb=accuracy_score(y_test,y_pred)
score_nb

0.8063033486539725

### SVM model

In [12]:
from sklearn.svm import SVC
svm=SVC(C=10,gamma=0.01,kernel='rbf')
svm.fit(x_vector,y_train)
y_pred=svm.predict(vectorizer.transform(x_test))
score_svm=accuracy_score(y_test,y_pred)
score_svm

0.8082731451083388

### From above all models Linear regression gives best accuracy of 81%

In [13]:
prediction=lr.predict(vectorizer.transform(test_df['text']))
submission=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
submission['target']=prediction
submission

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [14]:
submission.to_csv('submission.csv',index=False)